In [ ]:
import datajoint as dj
dj.config['database.host'] = 'datajoint.internationalbrainlab.org'

from ibl_pipeline import subject, acquisition, action, behavior, reference
from ibl_pipeline.analyses.behavior import PsychResults

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
myPath = r"C:\Users\Luigi\Documents\GitHub\ibl-changepoint\data" # Write here your data path
os.chdir(myPath)

In [ ]:
# We analyze data only from mice that have performed at least this number of stable sessions
MinSessionNumber = 5;

# These are the example mice
example_mice_nicknames = ['CSHL_005','CSHL_007','IBL-T1','IBL-T4','ibl_witten_04','ibl_witten_05']

# Here we download all mice
all_mice = subject.Subject().proj('subject_nickname')
mice_list = list(all_mice)
mice_names = list()
for subj in mice_list:
    mice_names.append(subj['subject_nickname'])

# Uncomment to only download the example mice
# mice_names = example_mice_nicknames
    
sess_stable   = (acquisition.Session & 'task_protocol LIKE "%biased%"')
stable_mice_names = list()

# Loop over all mice
for mouse_nickname in mice_names:
    
    mouse_subject = {'subject_nickname': mouse_nickname}
    
    # Have a look at the protocol for the first session (could probably be done in a smarter way)
    behavior_all = (acquisition.Session & (subject.Subject & mouse_subject)) \
        * subject.Subject.proj('subject_nickname') \
        * subject.SubjectLab.proj('lab_name')
    df0 = pd.DataFrame(behavior_all.fetch(order_by='session_start_time'))
    first_protocol = df0['task_protocol']

    # Only proceed with this mouse if it is a recent one (starts with a '*habituation*' protocol)
    if (len(first_protocol) > 0) and (first_protocol[0] is not None) and (first_protocol[0].lower().find('habituation') >= 0):

        # Get mouse data (thanks Anne Urai for the snippet)
        behavior_stable = (behavior.TrialSet.Trial & (subject.Subject & mouse_subject)) \
            * sess_stable.proj('session_uuid','task_protocol') * subject.Subject.proj('subject_nickname') \
            * subject.SubjectLab.proj('lab_name')
        df = pd.DataFrame(behavior_stable.fetch(order_by='subject_nickname, session_start_time, trial_id'))

        if len(df) > 0:           # The mouse has performed in at least one stable session with biased blocks

            position_deg = 35.    # Stimuli appear at +/- 35 degrees

            # Create new dataframe
            datamat = pd.DataFrame()
            datamat['trial_num'] = df['trial_id']
            datamat['session_num'] = np.cumsum(df['trial_id'] == 1)
            datamat['stim_probability_left'] = df['trial_stim_prob_left']
            signed_contrast = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']
            datamat['contrast'] = np.abs(signed_contrast)
            datamat['position'] = np.sign(signed_contrast)*position_deg
            datamat['response_choice'] = df['trial_response_choice']
            datamat.loc[df['trial_response_choice'] == 'CCW','response_choice'] = 1
            datamat.loc[df['trial_response_choice'] == 'CW','response_choice'] = -1
            datamat.loc[df['trial_response_choice'] == 'No Go','response_choice'] = 0
            datamat['trial_correct'] = np.double(df['trial_feedback_type']==1)
            datamat['reaction_time'] = df['trial_response_time'] - df['trial_stim_on_time'] # double-check

            # Since some trials have zero contrast, need to compute the alleged position separately
            datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'position'] = \
                datamat.loc[(datamat['trial_correct'] == 1) & (signed_contrast == 0),'response_choice']*position_deg
            datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'position'] = \
                datamat.loc[(datamat['trial_correct'] == 0) & (signed_contrast == 0),'response_choice']*(-position_deg)

            # Take mice that have performed a minimum number of sessions
            if np.max(datamat['session_num']) >= MinSessionNumber:

                # Save dataframe to CSV file
                filename = mouse_nickname + '.csv'
                datamat.to_csv(filename,index=False)

                stable_mice_names.append(mouse_nickname)

print(stable_mice_names)